In [ ]:
import cv2
import pathlib
import numpy as np
import matplotlib.pyplot as plt

%matplotlib widget
import seaborn as sns
import more_itertools as mit
from pathlib import Path
import os
from scipy import signal
import holoviews as hv
from holoviews import opts

import re
from tqdm import tqdm
import subprocess


hv.extension(
    "bokeh",
    #'matplotlib',
)

import sys

sys.path.insert(0, "..")

from Utilities.Utils import *
from Utilities.Processing import *

import black
import jupyter_black

jupyter_black.load()

# Test the process on one frame

In [ ]:
# Import the frame using opencv

frame = cv2.imread(
    "/mnt/labserver/DURRIEU_Matthias/Experimental_data/MultiMazeRecorder/Videos/Feeding_state/230704_FeedingState_1_AM/image0.jpg"
)

# make it grayscale
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# display the frame
plt.figure()
plt.imshow(frame, cmap="gray", vmin=0, vmax=255)
plt.show()

In [ ]:
# rotate the image 90 degrees
# frame = np.rot90(frame)

# display the frame
plt.figure()
plt.imshow(frame, cmap="gray", vmin=0, vmax=255)
plt.show()

In [ ]:
from PIL import Image


def generate_subsets(image_path, regions):
    """
    :param image_path: str, path to the image
    :param regions: list of tuples, each tuple contains 4 integers representing the coordinates of the region of interest in the form (left, upper, right, lower)
    :return: list of Image objects, each representing a subset of the main image
    """

    image = Image.open(image_path)

    # rotate the image 90 degrees
    image = image.rotate(90, expand=True)

    subsets = []
    for region in regions:
        subset = image.crop(region)
        subsets.append(subset)
    return subsets


# Example usage
regions_of_interest = [
    (0, 0, 850, 1000),
    (850, 0, 1900, 1000),
    (1900, 0, 2800, 1000),
    (0, 1000, 850, 2350),
    (850, 1000, 1900, 2350),
    (1900, 850, 2800, 2350),
    (0, 2350, 850, 3400),
    (850, 2350, 1900, 3400),
    (1900, 2350, 2800, 3400),
]
subsets = generate_subsets(
    "/home/matthias/Videos/Test2/image0.jpg", regions_of_interest
)

In [ ]:
# display all the subsets in a grid
fig, axs = plt.subplots(3, 3)
for i, ax in enumerate(axs.flat):
    ax.imshow(subsets[i])
plt.show()

In [ ]:
# generate a copy of the first subset as numpy array
subset = np.array(subsets[2])

In [ ]:
cols = subset.sum(axis=0)

hv.Histogram(cols).opts(tools=["hover"])

In [ ]:
peaks = signal.find_peaks(
    cols,
    distance=40,
    # height=(50_000, 150_000),
    # width=(10, 100),
    prominence=(20_000, 150_000),
)

# Check that peaks are correctly located

x = np.array(range(0, len(cols)))
PeaksPos = (x[peaks[0]], cols[peaks[0]])
hv.Histogram(cols).opts(tools=["hover"]) * hv.Points(PeaksPos).opts(
    color="orange", tools=["hover"]
)

In [ ]:
# Same but with another subset
subset = np.array(subsets[5])

cols = subset.sum(axis=0)


peaks = signal.find_peaks(
    cols,
    distance=40,
    height=(50_000, 300_000),
    # width=(10, 100),
    prominence=(20_000, 400_000),
)

# Check that peaks are correctly located

x = np.array(range(0, len(cols)))
PeaksPos = (x[peaks[0]], cols[peaks[0]])
hv.Histogram(cols).opts(tools=["hover"]) * hv.Points(PeaksPos).opts(
    color="orange", tools=["hover"]
)

In [ ]:
# for each subset, find the peaks and store their positions in a list
Arenapos = []
for subset in subsets:
    subset = np.array(subset)

    cols = subset.sum(axis=0)

    peaks = signal.find_peaks(
        cols,
        distance=40,
        height=(50_000, 300_000),
        prominence=(20_000, 400_000),
    )
    x = np.array(range(0, len(cols)))
    Arenapos.append(x[peaks[0]])

In [ ]:
Arenapos

In [ ]:
Arenapos[0]

In [ ]:
# Test cropping on a subset
ArenaList = []

sub = Arenapos[2]
for i in range(0, len(sub)):
    # if (i % 2) == 0:
    ArenaList.append(list(range(sub[i] - 45, sub[i] + 45)))

In [ ]:
%matplotlib inline

# Visual test if crop was successful

# convert subsets[0] to numpy array
testarray = np.array(subsets[2])
plt.imshow(testarray[:, ArenaList[2]], cmap="gray", vmin=0, vmax=255)

In [ ]:
# Detect vertical bounds
subset = np.array(subsets[4])

rows = subset.sum(axis=1)

hv.Histogram(rows).opts(tools=["hover"])

In [ ]:
# display the subset with only the rows where rows < 150_000
plt.imshow(subset[rows < 150_000], cmap="gray", vmin=0, vmax=255)

In [ ]:
plt.imshow(subset, cmap="gray", vmin=0, vmax=255)

That won't work because of the array heterogeneity. I need to crop the arenas properly right away

In [ ]:
# Reprocess the first frame
cols = frame.sum(axis=0)
hv.Histogram(cols).opts(tools=["hover"])

In [ ]:
rows = frame.sum(axis=1)
hv.Histogram(rows).opts(tools=["hover"])

In [ ]:
peaks = signal.find_peaks(
    cols,
    distance=1000,
    height=(200_000, 600_000),
    # width=(10, 100),
    # prominence=(20_000, 150_000),
)

# Check that peaks are correctly located

x = np.array(range(0, len(cols)))
PeaksPos = (x[peaks[0]], cols[peaks[0]])
hv.Histogram(cols).opts(tools=["hover"]) * hv.Points(PeaksPos).opts(
    color="orange", tools=["hover"]
)

In [ ]:
frame = cv2.imread("/home/matthias/Videos/Test2/image0.jpg")

# make it grayscale
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# equalize the histogram to make thresholding easier
frame = cv2.equalizeHist(frame)

# rotate the image 90 degrees
frame = np.rot90(frame)

# display the frame
plt.figure()
plt.imshow(frame, cmap="gray", vmin=0, vmax=255)
plt.show()

In [ ]:
Thresh = frame.copy()
# Apply a threshold to the frame to keep only the pixels with a value < 20
Thresh[Thresh > 60] = 0
# For the pixels with a value < 20, set their value to 255
Thresh[Thresh > 0] = 255
# display the frame
plt.imshow(Thresh, cmap="gray", vmin=0, vmax=255)

In [ ]:
plt.imshow(frame, cmap="gray", vmin=0, vmax=255)

In [ ]:
cols = Thresh.sum(axis=0)
hv.Histogram(cols).opts(tools=["hover"])

In [ ]:
rows = Thresh.sum(axis=1)
hv.Histogram(rows).opts(tools=["hover"])

In [ ]:
colbounds = signal.find_peaks(
    cols,
    distance=200,
    height=(300_000, 700_000),
)

# Check that peaks are correctly located

x = np.array(range(0, len(cols)))
ColBoundsPos = (x[colbounds[0]], cols[colbounds[0]])
hv.Histogram(cols).opts(tools=["hover"]) * hv.Points(ColBoundsPos).opts(
    color="orange", tools=["hover"]
)

In [ ]:
rowbounds = signal.find_peaks(
    rows,
    distance=200,
    height=(300_000, 700_000),
)

# Check that peaks are correctly located

x = np.array(range(0, len(rows)))
RowBoundsPos = (x[rowbounds[0]], rows[rowbounds[0]])
hv.Histogram(rows).opts(tools=["hover"]) * hv.Points(RowBoundsPos).opts(
    color="orange", tools=["hover"]
)

In [ ]:
def generate_subsets(image, regions):
    """
    :param image_path: str, path to the image
    :param regions: list of tuples, each tuple contains 4 integers representing the coordinates of the region of interest in the form (left, upper, right, lower)
    :return: list of numpy arrays, each representing a subset of the main image
    """
    subsets = []
    for region in regions:
        subset = image[region[1] : region[3], region[0] : region[2]]
        subsets.append(subset)
    return regions


# Example usage
xcoords = colbounds[0]
ycoords = rowbounds[0]

regions_of_interest = [
    (xcoords[0], ycoords[0], xcoords[1], ycoords[1]),
    (xcoords[2], ycoords[0], xcoords[3], ycoords[1]),
    (xcoords[4], ycoords[0], xcoords[5], ycoords[1]),
    (xcoords[0], ycoords[2], xcoords[1], ycoords[3]),
    (xcoords[2], ycoords[2], xcoords[3], ycoords[3]),
    (xcoords[4], ycoords[2], xcoords[5], ycoords[3]),
    (xcoords[0], ycoords[4], xcoords[1], ycoords[5]),
    (xcoords[2], ycoords[4], xcoords[3], ycoords[5]),
    (xcoords[4], ycoords[4], xcoords[5], ycoords[5]),
]
subsets = generate_subsets(frame, regions_of_interest)

In [ ]:
# Alt just keeping regions

regions_of_interest = [
    (xcoords[0], ycoords[0], xcoords[1], ycoords[1]),
    (xcoords[2], ycoords[0], xcoords[3], ycoords[1]),
    (xcoords[4], ycoords[0], xcoords[5], ycoords[1]),
    (xcoords[0], ycoords[2], xcoords[1], ycoords[3]),
    (xcoords[2], ycoords[2], xcoords[3], ycoords[3]),
    (xcoords[4], ycoords[2], xcoords[5], ycoords[3]),
    (xcoords[0], ycoords[4], xcoords[1], ycoords[5]),
    (xcoords[2], ycoords[4], xcoords[3], ycoords[5]),
    (xcoords[4], ycoords[4], xcoords[5], ycoords[5]),
]

In [ ]:
# display all the subsets in a grid
fig, axs = plt.subplots(3, 3)
for i, ax in enumerate(axs.flat):
    ax.imshow(subsets[i], cmap="gray", vmin=0, vmax=255)
plt.show()

In [ ]:
# Alt using the regions

fig, axs = plt.subplots(3, 3)
for i, ax in enumerate(axs.flat):
    ax.imshow(
        frame[
            regions_of_interest[i][1] : regions_of_interest[i][3],
            regions_of_interest[i][0] : regions_of_interest[i][2],
        ],
        cmap="gray",
        vmin=0,
        vmax=255,
    )
plt.show()

This is perfect ! I can now crop the arenas properly

In [ ]:
# for each subset, find the peaks and store their positions in a list
Arenapos = []
for subset in subsets:
    subset = np.array(subset)

    cols = subset.sum(axis=0)

    peaks = signal.find_peaks(
        cols,
        distance=40,
        height=(50_000, 300_000),
        prominence=(20_000, 400_000),
    )
    x = np.array(range(0, len(cols)))
    Arenapos.append(x[peaks[0]])

In [ ]:
ArenaList = []

sub = Arenapos[2]
for i in range(0, len(sub)):
    # if (i % 2) == 0:
    ArenaList.append(list(range(sub[i] - 40, sub[i] + 40)))

In [ ]:
%matplotlib inline
testarray = np.array(subsets[0])
# Reset plt and display only the next plot
plt.clf()

plt.figure()
plt.imshow(testarray[:, ArenaList[1]], cmap="gray", vmin=0, vmax=255)
plt.show()

Not ideal, what if I just apply the same method as done for the array?

In [ ]:
%matplotlib inline

# Test with one subset
testarray = np.array(subsets[3])

Thresh = testarray.copy()
# Apply an adaptive threshold to each subset to keep only the brightest pixels
Thresh = cv2.adaptiveThreshold(
    Thresh, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 31, 4
)

# display the frame

plt.imshow(Thresh, cmap="gray", vmin=0, vmax=255)

In [ ]:
# Same with the regions of interest

testarray = np.array(
    frame[
        regions_of_interest[3][1] : regions_of_interest[3][3],
        regions_of_interest[3][0] : regions_of_interest[3][2],
    ]
)

Thresh = testarray.copy()
# Apply an adaptive threshold to each subset to keep only the brightest pixels
Thresh = cv2.adaptiveThreshold(
    Thresh, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 31, 4
)

# display the frame

plt.imshow(Thresh, cmap="gray", vmin=0, vmax=255)

In [ ]:
%matplotlib inline
cols = Thresh.sum(axis=0)
plt.plot(cols)

In [ ]:
plt.plot(cols[100:620])

In [ ]:
# Detect peaks in the histogram of the subset
peaks = signal.find_peaks(
    cols[100:620],
    distance=20,
    height=(60_000, 120_000),
    # width=(2, 30),
    # prominence=(20_000, 400_000),
    # limit the x axis to the middle of the subset
)

# Add 100 to the x coordinates of the peaks to account for the offset
peaks = (peaks[0] + 100, peaks[1])


# plot cols with peaks points
x = np.array(range(0, len(cols)))
plt.plot(x, cols)
plt.plot(x[peaks[0]], cols[peaks[0]], "x")

In [ ]:
row = testarray.sum(axis=1)
plt.plot(row)

In [ ]:
# detect peaks in the histogram of the subset
peaks = signal.find_peaks(
    row,
    distance=300,
    # height=(60_000, 100_000),
    # prominence=(20_000, 400_000),
)

# plot cols with peaks points
x = np.array(range(0, len(row)))
plt.plot(x, row)
plt.plot(x[peaks[0]], row[peaks[0]], "x")

Example worked fine. Now I'll apply it to all the subsets.

In [ ]:
# For each subset, find the cols and rows peaks and store the even peaks in a list
Corridors = []
for i in range(len(regions_of_interest)):
    subset = np.array(
        frame[
            regions_of_interest[i][1] : regions_of_interest[i][3],
            regions_of_interest[i][0] : regions_of_interest[i][2],
        ]
    )
    Thresh = subset.copy()
    # Apply an adaptive threshold to each subset to keep only the brightest pixels
    Thresh = cv2.adaptiveThreshold(
        Thresh, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 31, 4
    )

    cols = Thresh.sum(axis=0)
    rows = subset.sum(axis=1)

    colpeaks = signal.find_peaks(
        cols[100:620],
        distance=20,
        height=(60_000, 120_000),
        # width=(5, 30),
    )
    colpeaks = (colpeaks[0] + 100, colpeaks[1])

    rowpeaks = signal.find_peaks(
        rows,
        distance=300,
    )
    #######################################################
    Colpos = []
    Rowpos = []

    for peak_index in colpeaks[0]:
        peak_x = regions_of_interest[i][0] + peak_index
        peak_y = np.argmax(subset[peak_index])
        peak_y += regions_of_interest[i][1]
        Colpos.append((peak_x, peak_y))

    for peak_index in rowpeaks[0]:
        peak_x = np.argmax(subset[peak_index])
        peak_x += regions_of_interest[i][0]
        peak_y = regions_of_interest[i][1] + peak_index
        Rowpos.append((peak_x, peak_y))

    bound_x = 30
    bound_y = 50

    subcors = [
        (
            Colpos[0][0] - bound_x,
            Rowpos[0][1] - bound_y,
            Colpos[1][0] + bound_x,
            Rowpos[1][1] + bound_y,
        ),
        (
            Colpos[2][0] - bound_x,
            Rowpos[0][1] - bound_y,
            Colpos[3][0] + bound_x,
            Rowpos[1][1] + bound_y,
        ),
        (
            Colpos[4][0] - bound_x,
            Rowpos[0][1] - bound_y,
            Colpos[5][0] + bound_x,
            Rowpos[1][1] + bound_y,
        ),
        (
            Colpos[6][0] - bound_x,
            Rowpos[0][1] - bound_y,
            Colpos[7][0] + bound_x,
            Rowpos[1][1] + bound_y,
        ),
        (
            Colpos[8][0] - bound_x,
            Rowpos[0][1] - bound_y,
            Colpos[9][0] + bound_x,
            Rowpos[1][1] + bound_y,
        ),
        (
            Colpos[10][0] - bound_x,
            Rowpos[0][1] - bound_y,
            Colpos[11][0] + bound_x,
            Rowpos[1][1] + bound_y,
        ),
    ]

    # subcors = generate_subsets(subset, regions_of_interest)

    Corridors.append(subcors)

In [ ]:
def peak_detection(image, regions):
    peaks = []
    for region in regions:
        x1, y1, x2, y2 = region
        sub_image = image[y1:y2, x1:x2]
        peak_indices = find_peaks(sub_image)
        for peak_index in peak_indices[0]:
            peak_x = x1 + peak_index
            peak_y = np.argmax(sub_image[peak_index])
            peak_y += y1
            peaks.append((peak_x, peak_y))
    return peaks

In [ ]:
# display all the corridors from all subsets in a grid
fig, axs = plt.subplots(3, 2)
for i, ax in enumerate(axs.flat):
    ax.imshow(Corridors[4][i], cmap="gray", vmin=0, vmax=255)

Now that we have the corridors coordinates, we can us them to write the videos.

In [ ]:
# display all the corridors from Corridors[0] in a grid based on the frame
fig, axs = plt.subplots(3, 2)
for i, ax in enumerate(axs.flat):
    ax.imshow(
        frame[
            Corridors[0][i][1] : Corridors[0][i][3],
            Corridors[0][i][0] : Corridors[0][i][2],
        ],
        cmap="gray",
        vmin=0,
        vmax=255,
    )

In [ ]:
plt.imshow(frame[Corridors[0][1]])

In [ ]:
# # Create a new directory for each corridor in each subset
# frame
# for i, subset in enumerate(Corridors):
#     for j, corridor in enumerate(subset):
#         os.makedirs(
#             f"/home/matthias/Videos/Test2_subsets/arena{i+1}/corridor_{j+1}", exist_ok=True
#         )
        
        
#         cv2.imwrite(
#             f"/home/matthias/Videos/Test2_subsets/arena{i+1}/corridor_{j+1}/corridor_{j}.png",
#             corridor,
#         )

In [ ]:
# Redo the same process but looping through all images in the Test2 folder

# Create a list of all the images in the Test2 folder
images = []
for filename in os.listdir("/home/matthias/Videos/Test2"):
    images.append(filename)

# sort the list of images by their number
images.sort(key=lambda f: int(re.sub("\D", "", f)))

In [ ]:
# # for each image, find the corridors and save them in the corresponding folder
# for i, image in enumerate(images):
#     frame = cv2.imread(f"/home/matthias/Videos/Test2/{image}")
#     frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     frame = np.rot90(frame)

#     for j, subset in enumerate(Corridors):
#         for k, corridor in enumerate(subset):
#             os.makedirs(
#                 f"/home/matthias/Videos/Test2_subsets/arena{j+1}/corridor_{k+1}",
#                 exist_ok=True,
#             )
#             cv2.imwrite(
#                 f"/home/matthias/Videos/Test2_subsets/arena{j+1}/corridor_{k+1}/{image}",
#                 corridor,
#             )

In [ ]:
# Same version but with a progress bar

# for i, image in tqdm(enumerate(images), total=len(images)):
#     frame = cv2.imread(f"/home/matthias/Videos/Test2/{image}")
#     frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     frame = np.rot90(frame)

#     for j, subset in enumerate(Corridors):
#         for k, corridor in enumerate(subset):
#             os.makedirs(
#                 f"/home/matthias/Videos/Test2_subsets/arena{j+1}/corridor_{k+1}",
#                 exist_ok=True,
#             )
#             # Write the image corresponding to the crop of the current frame
#             cv2.imwrite(
#                 f"/home/matthias/Videos/Test2_subsets/arena{j+1}/corridor_{k+1}/{image}",
#                 corridor,
#             )

In [ ]:
for i, image in tqdm(enumerate(images), total=len(images)):
    frame = cv2.imread(f"/home/matthias/Videos/Test2/{image}")
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame = np.rot90(frame)

    for j, subset in enumerate(Corridors):
        for k, corridor in enumerate(subset):
            # Create the subfolder if it doesn't exist
            subfolder = f"/home/matthias/Videos/Test2_subsets/arena{j+1}/corridor_{k+1}"
            os.makedirs(subfolder, exist_ok=True)

            # Crop the image
            x1, y1, x2, y2 = corridor
            cropped_image = frame[y1:y2, x1:x2]

            # Save the cropped image
            cropped_image_file = f"{os.path.splitext(image)[0]}_cropped.jpg"
            cv2.imwrite(os.path.join(subfolder, cropped_image_file), cropped_image)

In [ ]:
# define the path to the root folder containing the subfolders
root_folder = Path("/home/matthias/Videos/Test2_subsets")

# define the path to the output 'videos' folder
videos_folder = Path("/home/matthias/Videos/Test2_subsets/Videos")
videos_folder.mkdir(parents=True, exist_ok=True)

# define the output video parameters
fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # video codec
fps = 30  # frames per second

# iterate over the subfolders that are one level below the root folder
for subdir in root_folder.glob("*/*"):
    if not subdir.is_dir():
        continue

    # read the first image in the subfolder to determine the frame size
    first_image = cv2.imread(str(list(subdir.glob("*"))[0]))
    height, width, _ = first_image.shape
    frame_size = (width, height)

    # generate the output video filename from the folder and subfolder names
    relative_path = subdir.relative_to(root_folder)
    video_filename = videos_folder / f"{relative_path}.mp4"
    
    print(video_filename)

    # open a VideoWriter object for the current subfolder
    out = cv2.VideoWriter(str(video_filename), fourcc, fps, frame_size)

    # iterate over the images in the current subfolder
    for filename in tqdm(sorted(subdir.glob("*")), desc=str(subdir)):
        # read the image
        image = cv2.imread(str(filename))

        # convert the image from BGR to RGB color space
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # write the image to the video
        out.write(image)

    # release the VideoWriter object
    out.release()

In [ ]:
from pathlib import Path
import cv2
from tqdm import tqdm
import re

# define the path to the root folder containing the subfolders
root_folder = Path("/home/matthias/Videos/Test2_subsets")

# define the path to the output 'videos' folder
videos_folder = Path("/home/matthias/Videos/Test2_Videos/Videos_NumOrdered")
videos_folder.mkdir(parents=True, exist_ok=True)

# define the output video parameters
fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # video codec
fps = 30  # frames per second

# create a progress bar for the whole process
pbar = tqdm(total=len(list(root_folder.glob("*/*"))))


def sort_by_number(filename):
    # extract the number from the filename using a regular expression
    number = re.search(r"\d+", filename.name)
    if number:
        return int(number.group())
    else:
        return 0


# iterate over the subfolders that are one level below the root folder
for subdir in root_folder.glob("*/*"):
    if not subdir.is_dir():
        continue

    # read the first image in the subfolder to determine the frame size
    first_image_path = list(subdir.glob("*"))[0]
    first_image = cv2.imread(str(first_image_path))
    if first_image is None:
        print(f"Error: Failed to read image {first_image_path}")
        continue

    height, width, _ = first_image.shape
    frame_size = (width, height)

    # generate a unique output video filename from the subfolder name
    video_filename = videos_folder / f"{subdir.parent.name}_{subdir.name}.mp4"

    # open a VideoWriter object for the current subfolder
    out = cv2.VideoWriter(str(video_filename), fourcc, fps, frame_size)

    # iterate over the images in the current subfolder sorted by number
    for filename in sorted(subdir.glob("*"), key=sort_by_number):
        # read the image
        image = cv2.imread(str(filename))
        if image is None:
            print(f"Error: Failed to read image {filename}")
            continue

        # convert the image from BGR to RGB color space (if necessary)
        # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # write the image to the video
        out.write(image)

    # release the VideoWriter object
    out.release()

    # update the progress bar
    pbar.update(1)

# close the progress bar
pbar.close()

In [ ]:
# Try alternative method by writing videos with ffmpeg

# define the path to the root folder containing the subfolders
root_folder = Path("/home/matthias/Videos/Test2_subsets")

# define the path to the output 'videos' folder
videos_folder = Path("/home/matthias/Videos/Test2_subsets/Videos_ffmpeg")
videos_folder.mkdir(parents=True, exist_ok=True)

# create a progress bar for the whole process
pbar = tqdm(total=len(list(root_folder.glob("*/*"))))
# iterate over the subfolders that are one level below the root folder
for subdir in root_folder.glob("*/*"):
    if not subdir.is_dir():
        continue

    # generate a unique output video filename from the subfolder name
    video_filename = videos_folder / f"{subdir.name}.mp4"

    # generate the ffmpeg command
    command = [
        "ffmpeg",
        "-y",  # overwrite output files without asking
        "-framerate",
        "30",  # input frame rate
        "-pattern_type",
        "glob",  # use the glob pattern type
        "-i",
        f"{subdir}/*.jpg",  # input file pattern
        "-c:v",
        "libx264",  # use the h264 codec
        "-pix_fmt",
        "yuv420p",  # output pixel format
        "-crf",
        "17",  # constant rate factor
        "-vf",
        "pad=ceil(iw/2)*2:ceil(ih/2)*2",  # make the frame dimensions even
        str(video_filename),  # output video filename
    ]

    # run the ffmpeg command
    subprocess.run(command)

    # update the progress bar
    pbar.update(1)

# close the progress bar
pbar.close()

Grid writer

In [ ]:
import cv2
import numpy as np

# define the path to the input 'videos' folder
videos_folder = "/home/matthias/Videos/Test2_Videos/Videos_NumOrdered"

# define the path to the output 'grid' video
grid_video_filename = "/home/matthias/Videos/Test2_Videos/grid_video.mp4"

# define the output video parameters
fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # video codec
fps = 30  # frames per second

# define the grid size (number of rows and columns)
grid_rows = 6
grid_cols = 9

# initialize a list to store the VideoCapture objects for each input video
video_captures = []

# open all of the input videos and store their VideoCapture objects in the list
for y in range(1, grid_rows + 1):
    for x in range(1, grid_cols + 1):
        video_filename = f"{videos_folder}/arena{x}_corridor_{y}.mp4"
        cap = cv2.VideoCapture(video_filename)
        video_captures.append(cap)

# read the first frame from the first video to determine the frame size
ret, first_frame = video_captures[0].read()
if not ret:
    print("Error: Failed to read first frame from first video")
else:
    height, width, _ = first_frame.shape

    # calculate the size of the grid frame
    grid_frame_width = width * grid_cols
    grid_frame_height = height * grid_rows
    grid_frame_size = (grid_frame_width, grid_frame_height)

    # open a VideoWriter object for the output grid video
    out = cv2.VideoWriter(grid_video_filename, fourcc, fps, grid_frame_size)

    # initialize a variable to keep track of whether all videos have ended
    all_videos_ended = False

    # loop until all videos have ended
    while not all_videos_ended:
        # initialize an empty list to store the current frame from each video
        frames = []

        # read the current frame from each video
        for cap in video_captures:
            ret, frame = cap.read()
            if ret:
                # resize the frame to the common size if necessary
                if frame.shape[:2] != (height, width):
                    frame = cv2.resize(frame, (width, height))
                frames.append(frame)
            else:
                # if the video has ended, append a black frame
                black_frame = np.zeros((height, width, 3), dtype=np.uint8)
                frames.append(black_frame)

        # check if all videos have ended
        all_videos_ended = all(not cap.isOpened() for cap in video_captures)

        # create an empty grid frame
        grid_frame = np.zeros((grid_frame_height, grid_frame_width, 3), dtype=np.uint8)

        # copy each frame into its position in the grid frame
        for i, frame in enumerate(frames):
            row = i // grid_cols
            col = i % grid_cols
            x_start = col * width
            x_end = x_start + width
            y_start = row * height
            y_end = y_start + height
            grid_frame[y_start:y_end, x_start:x_end] = frame

        # write the grid frame to the output video
        out.write(grid_frame)

    # release all of the VideoCapture objects and the VideoWriter object
    for cap in video_captures:
        cap.release()
    out.release()